### Description
Scope: from observed data to set of 45h files with observed

As input is necessary a file of the station that cover the entire period. 

In [ ]:
wdir = "/home/daniele/documents/github/ftt01/phd/projects/hydrological_forecasting/"

In [ ]:
# IMPORTs
import os
import sys

# to link the lib in py scripts as well
os.chdir(wdir)
sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), 'lib'))

from lib import *

In [ ]:
import glob
from dateutil import tz
from shutil import rmtree

In [ ]:
### SETUP
# basins = ['passirio', 'plan']
basins = ['plan']
variables = ['temperature','precipitation','streamflow']

init_forecasting_hour = 9
lead_hours = 39

start_date_str = '20210615T00:00:00'
end_date_str = '20211016T00:00:00'
timezone_str = 'Europe/Rome'
timezone = ZoneInfo(timezone_str)

In [ ]:
start_date = dt.datetime.strptime(start_date_str, '%Y%m%dT%H:%M:%S').replace(tzinfo=ZoneInfo(timezone_str))
end_date = dt.datetime.strptime(end_date_str, '%Y%m%dT%H:%M:%S').replace(tzinfo=ZoneInfo(timezone_str))

dates = [ start_date + dt.timedelta(days=x) for x in range(0, (end_date-start_date).days) ]

In [ ]:
for variable in variables:
    for basin in basins:

        data_path = '/media/windows/projects/hydro_forecasting/machine_learning/data/observed/{basin}/{variable}/'
        output_path = "/media/windows/projects/hydro_forecasting/machine_learning/data/observed/{basin}/{variable}/daily/"
        
        data_path = data_path.format( basin=basin, variable=variable )
        output_path = output_path.format( basin=basin, variable=variable )
        mkNestedDir(output_path)

        stations = glob.glob( data_path + '*.txt' )

        for station in stations:

            print(station)
            
            station_data = pd.read_csv( station, parse_dates=[0], header=None, skiprows=4, index_col=0, names=['values'] )
            station_data.index.name = 'datetime'

            infer_dst = np.array([False] * station_data.shape[0])
            station_data = station_data.tz_localize(timezone_str, ambiguous=infer_dst)
            
            for date in dates:

                start_datetime = date + dt.timedelta( hours=init_forecasting_hour )
                end_datetime = date + dt.timedelta( hours=init_forecasting_hour + lead_hours )

                current_data = station_data[ start_datetime : end_datetime ]
                current_data.index = [ dt.datetime.strftime(i, format='%Y-%m-%d %H:%M:%S') for i in current_data.index ]
                current_data.index.name = "datetime"
                
                current_data.to_csv( output_path + dt.datetime.strftime(start_datetime, format='%Y%m%d') + '.csv', sep=';' )